# Field Apps Deployment using Python

<img src="https://www.esri.com/content/dam/esrisites/en-us/common/icons/product-logos/arcgis-field-maps-220.png" width="100">

#### Getting your ArcGIS Field Maps workers set up for an event can be a time-consuming and mistake-ridden process. In this demo, we're going to show how you can utilize the ArcGIS API for Python to improve the speed and accuracy of setup for three different forms of data collection:

Note: this requires the ArcGIS API for Python 2.4.0 (released Oct 2024). Much of this notebook also requires having admin privileges in your organization. This notebook was originally created as part of DevSummit 2021.


1. Location Tracking (Enable LTS, Create Track View, Assign Licenses)
2. Forms data collection (Create Map, Create Form, Copy Form Between Maps, QR code generation)
3. Workforce coordination (Create a Project, Create Assignment Types, Add Workers, Add Assignments)



## Location Tracking

Location tracking is a feature that enables organizations to capture the tracks of field staff, monitor where they are, and analyze where they have been. Patterns derived from location tracks give decision-makers information to support critical field activities.

Tracks can be collected either via ArcGIS Field Maps or ArcGIS Tracker if your organization has purchased ArcGIS Tracker licenses and field workers have been assigned those licenses. Since setting up licenses and track views for a group of workers may be a time-consuming task, let's use the ArcGIS API for Python `arcgis.apps.tracker` [module](https://developers.arcgis.com/python/api-reference/arcgis.apps.tracker.html) to speed up this workflow!

In order to start location tracking, let's make sure it's enabled at an organizational level for my org.

In [ ]:
# Enable Location Tracking

import arcgis
from arcgis.map import Map
from arcgis.map.forms import FormGroupElement
from arcgis.map.forms import FormFieldElement
from arcgis.map.forms import FormTextBoxInput
from arcgis.map.forms import FormInfo
from arcgis.features import FeatureLayer
from arcgis.layers import Service

gis = arcgis.gis.GIS("<org_url>", "<insert_user_here>", verify_cert=False)
status = gis.admin.location_tracking.status
print("Location Tracking Status: " + status)
print("Enabling")
if status == "disabled":
    gis.admin.location_tracking.enable()
print("New Location Tracking Status: " + gis.admin.location_tracking.status)

#### Assign ArcGIS Tracker licenses

In order to activate tracking in ArcGIS Tracker or Field Maps, ArcGIS Tracker licenses must be assigned to workers. This can be a time consuming process, but it's quite simple in Python.

In [ ]:
# Assign Tracker licenses to group users

group = gis.groups.search("workers")[0]
print(group)
members = group.get_members()
print(members)
print("\n")
tracker_license = gis.admin.license.get("arcgis tracker")
for user in members["users"]:
    tracker_license.assign(user, ["tracker"])
print("Assigned!")

#### Now, let's pull all of our users into a track view - that will make visualizing tracks through ArcGIS Track Viewer much simpler

To make a track view, we can also utilize the `tracker` [module](https://developers.arcgis.com/python/api-reference/arcgis.apps.tracker.html) here.

In [ ]:
# Add Tracker users from same group to new track view as mobile users

track_view = gis.admin.location_tracking.create_track_view("workers_view")
track_view.mobile_users.add(members["users"])
print("Number of users in track view: " + str(len(track_view.mobile_users.list())))

#### Finally, let's generate an easy QR code using the Python QRCode library that my field workers can scan in order to login and begin tracking

Scanning this and logging into Field Maps will turn on tracking for a 10 hour duration. For more on our app linking parameters, our documentation can be found [here](https://doc.arcgis.com/en/field-maps/android/help/deploy-your-map.htm). For more location tracking usecases, please visit our Tracker Github [repository](https://github.com/Esri/tracker-scripts)

In [ ]:
import qrcode

begin_tracking_url = "https://fieldmaps.arcgis.app/?referenceContext=track&trackingOn=true&trackingDuration=10"
img = qrcode.make(begin_tracking_url)
img

## Forms Data Collection

Smart forms are the new way to edit non-spatial information about GIS features in ArcGIS Field Maps. Forms are built upon existing, published web feature layers and the form you create is stored in your map. Smart forms allow much more customization of the data collection experience compared to older workflows.

While most users will create their smart forms using the ArcGIS Field Maps web app, there may be situations where a GIS administrator needs to batch create a variety of forms in an efficient manner. In this instance, the `arcgis.map.forms.FormInfo` and `arcgis.map.map_widget.MapContent`(being released at ArcGIS API for Python version 2.4.0) is a great tool for you.

In this particular example, I'll create a smart form for a publicly available emergency shelters layer. Notice how I create my own form for a layer I do not own, since the form information resides in my webmap rather than their layer.

In [ ]:
# Create Map with new feature layer
wm = gis.map()
wm.basemap.basemap = "arcgis-streets"
wm.center = [41.75, -88]
wm.zoom = 10
wm

In [ ]:
item_properties = {
    "title": "Shelters Form Map",
    "snippet": "Map for shelters",
    "tags": ["shelters"],
}
wm = wm.save(item_properties)
newwm = Map(item=wm)
newwm.content.add(
    FeatureLayer(
        "https://sampleserver6.arcgisonline.com/arcgis/rest/services/NapervilleShelters/FeatureServer/0"
    ),
)
newwm.update()

In [ ]:
field_elements = [
    FormFieldElement(
        field_name="facilityid",
        input_type=FormTextBoxInput(),
        label="Facility ID",
        editable_expression="expr/system/true",
    ),
    FormFieldElement(
        field_name="jurisdict",
        input_type=FormTextBoxInput(),
        label="New Jursdiction",
        editable_expression="expr/system/true",
    ),
]

form_info = FormInfo(title="Shelters Layer", form_elements=field_elements)
newwm.content.update_layer(0, form=form_info)
newwm.content.form(0)
newwm.update()

print(newwm.content.form(0))

### We can easily populate a form using the `FormFieldElements`, `FormGroupElement` and `FormInfo` classes - this allows us to bypass the web application and perform all operations in Python

I can do the same sorts of actions present in the Field Maps web app - adding groups, changing form properties, and add the same types of conditional visibility expressions to ensure the form element "New Jurisdiction" only shows up if "Facility ID" is equal to a certain value

In [10]:
field_elements = [
    FormFieldElement(
        field_name="pocname",
        input_type=FormTextBoxInput(),
        label="Contact Name",
        editable_expression="expr/system/true",
    ),
    FormFieldElement(
        field_name="pocemail",
        input_type=FormTextBoxInput(),
        label="Contact Email",
        editable_expression="expr/system/true",
    ),
]

In [11]:
group_element = [
    FormGroupElement(
        title="Contact Information",
        label="Contact Information",
        initial_state="expanded",
        form_elements=field_elements,
        visibility_expression="$feature.facilityid == '1'",
    )
]

In [ ]:
form_info = FormInfo(title="Contact Info", form_elements=group_element)
newwm.content.update_layer(0, form=form_info)
newwm.content.form(0)
newwm.update()
print(newwm.content.form(0))

### Batch operations with forms

We could use this Python module to perform batch operations with forms that may be slower in the web application. For example, let's say I now want to use this same form in a different map that uses the same operational layer. This is relatively easily when given the forms module!

For more on what you can accomplish with the forms module, please visit our [documentation](https://developers.arcgis.com/python/api-reference/arcgis.mapping.forms.html)

In [ ]:
wm2 = arcgis.map.Map(gis.content.get("ae8b773ff6474e5993ed49e3e798e3d4"))
print(wm2.content.form(0))

In [ ]:
new_form = wm2.content.form(0)
new_form = form_info
print(new_form)
wm2.update()

## Workforce Coordination

ArcGIS Workforce is a mobile solution that uses the power of location-based decision making for better mobile workforce coordination and teamwork. It's composed of a web app used by project administrators and dispatchers in the office, and a mobile app used by mobile workers on their devices.

Users may want to utilize the ArcGIS API for Python `workforce` module to automate tasks. Automation is especially useful when managing Workforce projects that contain a large number of workers and assignments. Import workers from a CSV file, assign work, create a dashboard, delete assignments, reset worker status, and more with ArcGIS API for Python.

Note here that you can use the Python API to create your "classic" Workforce projects which work with the older version of the Workforce mobile application.

In [10]:
# create new project, create "classic" workforce project

new_proj = arcgis.apps.workforce.create_project("new_project", major_version=2)
old_proj = arcgis.apps.workforce.create_project("old_project", major_version=1)
print(new_proj)
print(old_proj)

new_project
old_project


#### Project Setup

Now that we've got our project, we're ready to get it setup for automation! Workforce projects require both assignment types to categorize your assignments and workers to send assignments. Using the group we already have from earlier in the demo, let's get our workers into the project and ready to work.

In [ ]:
print("Adding assignment types and workers")
at = new_proj.assignment_types.add(name="Demolition")
for user in members["users"]:
    new_proj.workers.add(user_id=user, name=user)
print("Assignment types and workers added")

Adding assignment types and workers
Assignment types and workers added


In [ ]:
# Add an assignment

new_proj.assignments.add(
    assignment_type=at,
    location="100 Commercial St",
    geometry={"x": -7820308, "y": 5412450},
    status=0,
    description="hello",
)

<Assignment 1>

### Resources

This is just a brief introduction into what is possible with task and data management using the ArcGIS API for Python. We have lots of examples available to learn how to write your own scripts and customize these notebooks as you see fit. Here's a non-comprehensive list of resources available:

- [ArcGIS API for Python homepage](https://developers.arcgis.com/python/)
- [API documentation](https://developers.arcgis.com/python/api-reference/)
- [Workforce Scripts](https://github.com/Esri/workforce-scripts)
- [Tracker Scripts](https://github.com/Esri/tracker-scripts)
- [Automating Workforce with Python Blog](https://www.esri.com/arcgis-blog/products/workforce/field-mobility/automate-workforce-with-arcgis-api-for-python/)
